In [5]:
%load_ext autoreload
%autoreload 2
import os
import sys
import numpy as np
import pandas as pd
import xarray as xr
import pingouin as pg
import plotly.graph_objects as go
from os.path import join as pjoin
from natsort import natsorted

sys.path.append("../../")
import circletrack_behavior as ctb
import plotting_functions as pf

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [59]:
## Settings
parent_dir = 'CircleTrack_MultiPort'
experiment_dir = 'MultiPort1'
lin_path = f'../../../{parent_dir}/{experiment_dir}/output/lin_behav/'
circle_path = f'../../../{parent_dir}/{experiment_dir}/output/behav/'
fig_path = f'../../../{parent_dir}/{experiment_dir}/intermediate_figures'
chance_color = 'darkgrey'
avg_color = 'midnightblue'
subject_color = 'darkgrey'
mice_list = [f'mp0{x}' for x in np.arange(1, 10)] + ['mp10', 'mp11', 'mp12']
two_port = ['mp01', 'mp04', 'mp07', 'mp10']
four_port = ['mp02', 'mp05', 'mp08', 'mp11']
group_colors_dict = {'Two Port': 'darkgrey', 'Four Port': 'midnightblue', 'Six Port': 'darkorchid'}
group_colors_list = ['midnightblue', 'darkorchid', 'darkgrey']
error_color = ['rgba(169,169,169,0.4)', 'rgba(25,25,112,0.4)', 'rgba(153,50,204,0.4)']
group_list = ['Two Port', 'Four Port', 'Six Port']

if not os.path.exists(fig_path):
    os.makedirs(fig_path)

### Plot linear track rewards across days.

In [ ]:
lin_dict = {'mouse': [], 'day': [], 'rewards': [], 'group': [], 'group_two': []}
for mouse in os.listdir(lin_path):
    mouse_path = pjoin(lin_path, mouse)
    group = 'Two Port' if mouse in two_port else 'Four Port' if mouse in four_port else 'Six Port'
    for idx, session in enumerate(os.listdir(mouse_path)):
        lin_behav = pd.read_feather(pjoin(mouse_path, f'{session}'))
        lin_dict['mouse'].append(mouse)
        lin_dict['day'].append(idx+1)
        lin_dict['rewards'].append(np.sum(lin_behav['water']))
        lin_dict['group'].append(group)
        lin_dict['group_two'].append(lin_behav['maze'].unique()[0])
lin_df = pd.DataFrame(lin_dict)

In [ ]:
## Plot rewards across days on linear track
fig = pf.plot_behavior_across_days(lin_df, x_var='day', y_var='rewards', groupby_var=['day'], plot_transitions=None,
                                   marker_color=subject_color, avg_color=avg_color, expert_line=False, chance=False, plot_datapoints=False,
                                   x_title='Day', y_title='Rewards', titles=['Linear Track'], height=500, width=500)
fig.show()
fig.write_image(pjoin(fig_path, 'linear_track_rewards.png'))

### Circle track lick accuracy

In [42]:
circletrack_results = {'mouse': [], 'day': [], 'group': [], 'session': [], 'lick_accuracy_thresh5': [], 'rewards': [], 'num_trials': []}
for mouse in os.listdir(circle_path):
    mouse_path = pjoin(circle_path, mouse)
    group = 'Two Port' if mouse in two_port else 'Four Port' if mouse in four_port else 'Six Port'
    for idx, session in enumerate(os.listdir(mouse_path)):
        behav = pd.read_feather(pjoin(mouse_path, f'{session}'))
        behav = behav[~behav['probe']]
        if group == 'Two Port':
            reward_one, reward_two = np.unique(behav['reward_one'])[0], np.unique(behav['reward_two'])[0]
            rewards = [reward_one, reward_two]
        elif group == 'Four Port':
            reward_one, reward_two = np.unique(behav['reward_one'])[0], np.unique(behav['reward_two'])[0]
            reward_three, reward_four = np.unique(behav['reward_three'])[0], np.unique(behav['reward_four'])[0]
            rewards = [reward_one, reward_two, reward_three, reward_four]
        elif group == 'Six Port':
            reward_one, reward_two = np.unique(behav['reward_one'])[0], np.unique(behav['reward_two'])[0]
            reward_three, reward_four = np.unique(behav['reward_three'])[0], np.unique(behav['reward_four'])[0]
            reward_five, reward_six = np.unique(behav['reward_five'])[0], np.unique(behav['reward_six'])[0]
            rewards = [reward_one, reward_two, reward_three, reward_four, reward_five, reward_six]
        
        pc = ctb.lick_accuracy(behav, port_list=rewards, lick_threshold=5, by_trials=False)
        circletrack_results['mouse'].append(mouse)
        circletrack_results['day'].append(idx+1)
        circletrack_results['group'].append(group)
        circletrack_results['session'].append(np.unique(behav['session_two'])[0])
        circletrack_results['lick_accuracy_thresh5'].append(pc)
        circletrack_results['rewards'].append(np.sum(behav['water']))
        circletrack_results['num_trials'].append(behav['trials'].unique()[-1])
ct_df = pd.DataFrame(circletrack_results)

In [ ]:
## Plot 5th lick accuracy across days
fig = pf.plot_behavior_across_days(ct_df[ct_df['day'] < 6], x_var='day', y_var='lick_accuracy_thresh5', groupby_var=['day', 'group'], plot_transitions=None,
                                   marker_color=group_colors_list, avg_color=avg_color, expert_line=False, chance=True, plot_datapoints=False,
                                   x_title='Day', y_title='Lick Accuracy (%)', titles=['Circle Track'], height=500, width=500)

fig.add_hline(y=50, line_width=1, line_dash='dash', line_color='midnightblue', opacity=1)
fig.add_hline(y=75, line_width=1, line_dash='dash', line_color='darkorchid', opacity=1)
fig.update_yaxes(range=[0, 100])
fig.show()
fig.write_image(pjoin(fig_path, 'lick_accuracy.png'))

In [ ]:
## Plot rewards across days
fig = pf.plot_behavior_across_days(ct_df[ct_df['day'] < 6], x_var='day', y_var='rewards', groupby_var=['day', 'group'], plot_transitions=None,
                                   marker_color=group_colors_list, avg_color=avg_color, expert_line=False, chance=False, plot_datapoints=False,
                                   x_title='Day', y_title='Rewards', titles=['Circle Track'], height=500, width=500)
fig.update_yaxes(range=[0, 120])
fig.show()
fig.write_image(pjoin(fig_path, 'circle_track_rewards.png'))

In [ ]:
## Plot number of trials
fig = pf.custom_graph_template(x_title='Day', y_title='Number of Trials', width=1000)
for day in ct_df['day'].unique():
    for group in group_list:
        data = ct_df[(ct_df['day'] == day) & (ct_df['group'] == group)]
        fig.add_trace(go.Violin(x=data['day'], y=data['num_trials'], box_visible=True, meanline_visible=True, name=group,
                                legendgroup=str(day), scalegroup=str(day), line_color=group_colors_dict[group], showlegend=False))
fig.update_traces(points='all', jitter=0.4)
fig.update_layout(violinmode='group')
fig.show()

In [62]:
## Plot number of trials
avg_trials = ct_df.groupby(['group', 'day'], as_index=False).agg({'num_trials': ['mean', 'sem']})
fig = pf.custom_graph_template(x_title='Day', y_title='Number of Trials')
for group in group_list:
    gdata = avg_trials[avg_trials['group'] == group]
    fig.add_trace(go.Scatter(x=gdata['day'], y=gdata['num_trials']['mean'], line_color=group_colors_dict[group],
                             name=group, error_y=dict(type='data', array=gdata['num_trials']['sem'])))
fig.update_yaxes(range=[0, 60])
fig.show()

In [64]:
ct_df.mixed_anova(dv='num_trials', within='day', between='group', subject='mouse')

,Source,SS,DF1,DF2,MS,F,p-unc,np2,eps
0,group,3649.845455,2,8,1824.922727,5.813939,0.027597,0.592416,NaN
1,day,1073.163636,4,32,268.290909,4.574603,0.004913,0.363797,0.629409
2,Interaction,912.503030,8,32,114.062879,1.944875,0.087110,0.327152,NaN


### Lick accuracy across trials for all groups.

In [10]:
bin_size = 3
trial_res = {'mouse': [], 'day': [], 'group': [], 'session': [], 'trial': [], 'lick_acc': []}
for mouse in os.listdir(circle_path):
    mouse_path = pjoin(circle_path, mouse)
    group = 'Two Port' if mouse in two_port else 'Four Port' if mouse in four_port else 'Six Port'
    for idx, session in enumerate(os.listdir(mouse_path)):
        behav = pd.read_feather(pjoin(mouse_path, f'{session}'))
        behav = behav[~behav['probe']]
        if group == 'Two Port':
            reward_one, reward_two = np.unique(behav['reward_one'])[0], np.unique(behav['reward_two'])[0]
            rewards = [reward_one, reward_two]
        elif group == 'Four Port':
            reward_one, reward_two = np.unique(behav['reward_one'])[0], np.unique(behav['reward_two'])[0]
            reward_three, reward_four = np.unique(behav['reward_three'])[0], np.unique(behav['reward_four'])[0]
            rewards = [reward_one, reward_two, reward_three, reward_four]
        elif group == 'Six Port':
            reward_one, reward_two = np.unique(behav['reward_one'])[0], np.unique(behav['reward_two'])[0]
            reward_three, reward_four = np.unique(behav['reward_three'])[0], np.unique(behav['reward_four'])[0]
            reward_five, reward_six = np.unique(behav['reward_five'])[0], np.unique(behav['reward_six'])[0]
            rewards = [reward_one, reward_two, reward_three, reward_four, reward_five, reward_six]
        
        trial_acc = ctb.lick_accuracy(behav, port_list=rewards, lick_threshold=5, by_trials=True)

        if bin_size > 0:
            binned_acc = ctb.bin_data(trial_acc, bin_size)
        else:
            binned_acc = trial_acc
        
        for trial, val in enumerate(binned_acc):
            trial_res['mouse'].append(mouse)
            trial_res['group'].append(group)
            trial_res['day'].append(idx+1)
            trial_res['session'].append(behav['session_two'].unique()[0])
            trial_res['trial'].append(trial+1)
            trial_res['lick_acc'].append(val)
trial_df = pd.DataFrame(trial_res)
avg_acc = trial_df.groupby(['group', 'day', 'session', 'trial'], as_index=False).agg({'lick_acc': ['mean', 'sem']})

/media/caishuman/csstorage3/Austin/CircleTrack/CircleTrackAnalysis/Circletrack_MultiPort/MultiPort1/../../circletrack_behavior.py:532: RuntimeWarning: Mean of empty slice
  return [np.nanmean(bin) for bin in binned if bin.size > 0]
/media/caishuman/csstorage3/Austin/CircleTrack/CircleTrackAnalysis/Circletrack_MultiPort/MultiPort1/../../circletrack_behavior.py:532: RuntimeWarning: Mean of empty slice
  return [np.nanmean(bin) for bin in binned if bin.size > 0]
/media/caishuman/csstorage3/Austin/CircleTrack/CircleTrackAnalysis/Circletrack_MultiPort/MultiPort1/../../circletrack_behavior.py:532: RuntimeWarning: Mean of empty slice
  return [np.nanmean(bin) for bin in binned if bin.size > 0]
/media/caishuman/csstorage3/Austin/CircleTrack/CircleTrackAnalysis/Circletrack_MultiPort/MultiPort1/../../circletrack_behavior.py:532: RuntimeWarning: Mean of empty slice
  return [np.nanmean(bin) for bin in binned if bin.size > 0]
/media/caishuman/csstorage3/Austin/CircleTrack/CircleTrackAnalysis/Circl

In [34]:
fig = pf.custom_graph_template(x_title='', y_title='', rows=1, columns=5, shared_y=True, shared_x=True, width=1200)
for gind, group in enumerate(['Two Port', 'Four Port', 'Six Port']):
    gdata = avg_acc[avg_acc['group'] == group]
    for idx, day in enumerate(gdata['day'].unique()):
        day_data = gdata[gdata['day'] == day]
        row, col = 1, idx + 1
        x_data = np.arange(1, np.array(day_data['trial'])[-1]*bin_size, bin_size)
        upper = day_data['lick_acc']['mean'] + day_data['lick_acc']['sem']
        lower = day_data['lick_acc']['mean'] - day_data['lick_acc']['sem']

        fig.add_trace(go.Scatter(x=x_data, y=day_data['lick_acc']['mean'], mode='lines', line_color=group_colors_dict[group], showlegend=False,
                                 name=group, legendgroup=group), row=row, col=col)
        fig.add_trace(go.Scatter(x=x_data, y=upper, mode='lines', marker=dict(color=error_color[gind]),
                                name='Upper Bound', line=dict(width=0), showlegend=False), row=row, col=col)
        fig.add_trace(go.Scatter(x=x_data, y=lower, mode='lines', marker=dict(color=error_color[gind]),
                                name='Lower Bound', line=dict(width=0), showlegend=False, fillcolor=error_color[gind], fill='tonexty'), row=row, col=col)
                             
fig.add_hline(y=25, line_width=1, line_dash='dash', line_color=chance_color, opacity=1)
fig.add_hline(y=50, line_width=1, line_dash='dash', line_color='midnightblue', opacity=1)
fig.add_hline(y=75, line_width=1, line_dash='dash', line_color='darkorchid', opacity=1)
fig.update_yaxes(title='Lick Accuracy (%)', col=1)
fig.update_xaxes(title='Trial', row=4)
fig.update_yaxes(range=[0, 101])
fig.data[0]['showlegend'] = True
fig.data[15]['showlegend'] = True
fig.data[30]['showlegend'] = True
fig.show()